In [1]:
import joblib
import pytesseract
import cv2
import pickle
from keras import backend as K
import joblib
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import models,layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import os
import streamlit as st

In [2]:
OrganModel=joblib.load('/Users/rahul/Downloads/brain_tumor_dataset/organ_clissification')
ChestModel=joblib.load('/Users/rahul/Downloads/brain_tumor_dataset/chestClassification')
BrainModel=joblib.load('/Users/rahul/Downloads/brain_tumor_dataset/brain_classification')
KneeModel=joblib.load('/Users/rahul/Downloads/brain_tumor_dataset/knee_prediction_model')
BrainModel3=joblib.load('/Users/rahul/Downloads/brain_tumor_dataset/brain_tumour_model')
HandFractureModel=joblib.load('/Users/rahul/Downloads/brain_tumor_dataset/handFracture_prediction_model')

In [ ]:
organ_class_indices=['brain','chest','hand_fracture','knee']
brain_class_indices=['no','yes']
brain_classes_3=['glioma', 'meningioma', 'no', 'pituitary']
chest_class_indices=['no','yes']
hand_fracture_indices=['no','yes']
indexKnee_label_dict = {
            0: 'Disease',
            1: 'no',
}

In [ ]:
def preprocess(img):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(224,224))
    print(img.shape)
    plt.imshow(img,cmap=plt.cm.binary)

    prediction=model.predict(np.array([img])/255)
    index=np.argmax(prediction)
    print(classes_name[index])

In [ ]:
def preprocess_brain_3(img):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(245,256))
    print(img.shape)
    plt.imshow(img,cmap=plt.cm.binary)

    prediction=model.predict(np.array([img])/255)
    index=np.argmax(prediction)
    print(classes_name[index])

In [ ]:
st.title("DISEASE CLASSIFICATION")

In [ ]:
uploaded_file = st.file_uploader("Choose an image...", type=["png", "jpg", "jpeg"])

In [ ]:
from PIL import Image

In [ ]:
def load_image_into_cv2(uploaded_file):
    # Convert the uploaded file to an OpenCV image
    image = Image.open(uploaded_file)
    image_array = np.array(image)
    image_cv = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    return image_cv

In [ ]:
def chest_prediction(img):
    prediction=ChestModel.predict(np.array([img])/255)
    index=np.argmax(prediction)
    p1=chest_class_indices[index]
    return p1

In [ ]:
def brain_prediction(img):
    prediction=BrainModel.predict(np.array([img])/255)
    index=np.argmax(prediction)
    p1=brain_class_indices[index]
    return p1

In [ ]:
def brain_prediction_3(img):
    img=cv2.resize(img,(256,256))
    prediction=BrainModel3.predict(np.array([img])/255)
    index=np.argmax(prediction)
    p1=brain_classes_3[index]
    return p1

In [ ]:
def knee_prediction(img):
    img=cv2.resize(img,(200,200))
    prediction=KneeModel.predict(np.array([img])/255)
    index=np.argmax(prediction)
    p1= indexKnee_label_dict.get(index, "Unknown")
    return p1

In [ ]:
def hand_prediction(img):
    img=cv2.resize(img,(200,200))
    prediction=HandFractureModel.predict(np.array([img])/255)
    index=np.argmax(prediction)
    p1= indexKnee_label_dict.get(index, "Unknown")
    return p1

In [ ]:
if uploaded_file is not None:
    
    # Load the image into OpenCV format
    img = load_image_into_cv2(uploaded_file)
    
    # Display the uploaded image using Streamlit
    st.image(img, caption='Uploaded Image (OpenCV format)', use_column_width=True, channels='BGR')
    
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(224,224))
    print(img.shape)
    plt.imshow(img,cmap=plt.cm.binary)

    prediction=OrganModel.predict(np.array([img])/255)
    index=np.argmax(prediction)
    organP=organ_class_indices[index]

    #if the image is a chest image
    
    if organP=='chest':
        Org='The given X-ray image is a Chest MRI'
        st.write(f"Prediction: {Org}")
        p1=chest_prediction(img)
        if p1=='no':
            p1='no pneumonia'
            st.write(f"Prediction: {p1}")
        else:
            p1='penumonia'
            st.write(f"Prediction: {p1}")

    #if the image is a brain image        
    
    if organP=='brain':
        Org='The given MRI ray image is a brain MRI'
        st.write(f"Prediction: {Org}")
        p1=brain_prediction(img)
        p2=brain_prediction_3(img)
        if p1=='no' and p2=='no':
            p1='no brain tumour'
            st.write(f"Prediction: {p1}")
        elif p1=='yes' and p2=='no':
            p1='Doubtful regarding brain tumour'
            st.write(f"Prediction: {p1}")
        else:
            st.write(f"There is Brain tumor")
            st.write(f"Prediction: {p2}")
            
    if organP=='knee':
        Org='The given X-ray image is a Knee X-ray'
        st.write(f"Prediction: {Org}")
        p1=knee_prediction(img)
        if p1=='no':
            p1='no knee osteoporosis'
            st.write(f"Prediction: {p1}")
        else:
            p1='There is knee osteoporosis'
            st.write(f"Prediction: {p1}")
            
    if organP=='hand_fracture':
        Org='The given X-ray image is a Hand X-ray'
        st.write(f"Prediction: {Org}")
        p1=hand_prediction(img)
        if p1=='no':
            p1='no hand fracture'
            st.write(f"Prediction: {p1}")
        else:
            p1='There is hand fracture'
            st.write(f"Prediction: {p1}")

    #if the image is a brain image